In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
from fastparquet import write
import fastparquet
import multiprocessing as mp
import snappy
import argparse
import multiprocessing as mp
from multiprocessing import Pool

In [2]:
parser = argparse.ArgumentParser(description='Program to combine pickle data into parquet files for version 3')


parser.add_argument('--spath', type=str,
                    help='directory where to save output')
parser.add_argument('--dec', type=str,
                    help='Which decade to be processed?')

args = parser.parse_args('--spath /datanaco/dharp/compounds/datasets/googleV3/ --dec 2010'.split())


In [3]:
save_path=args.spath

In [4]:
pkl_files=glob.glob(f'{args.spath}/{args.dec}/*pkl')
len(pkl_files)

10333

In [5]:
def df_list_processor(df_list):
    
    if type(df_list)==str:
        df_list=[df_list]
    concat_list=[]
    for cur_file in df_list:
        #print(row.fname)
        cur_df=pd.read_pickle(cur_file)
        concat_list.append(cur_df)
    concat_df=pd.concat(concat_list,ignore_index=True,sort=True)
    total_df_shape=concat_df.shape[0]
    
    new_df=concat_df.groupby(['lemma_pos','comp_class','is_comp','comp_ner_sent'])['count'].sum().to_frame().reset_index()
    
    new_df["comp_class"] = new_df["comp_class"].astype("category")
    new_df["comp_ner_sent"] = new_df["comp_ner_sent"].astype("category")
    
    return new_df

In [6]:
print(f'Decade {args.dec}')

num_partitions=round(0.95*mp.cpu_count())
cur_time=time.time()
pool = Pool(num_partitions)
print('Started parallelization')
results=pool.map_async(df_list_processor,pkl_files,chunksize=len(pkl_files)//num_partitions)
pool.close()
pool.join()
    
print('Done parallelization')

curr_df_list=results.get()
new_index_df=pd.concat(curr_df_list,ignore_index=True)
old_shape=new_index_df.shape[0]

#new_index_df=new_index_df.groupby(['lemma_pos','comp_class','is_comp','comp_ner_sent'],observed=True)['count'].sum().to_frame().reset_index()

#print(f'{new_index_df.shape[0]/old_shape}')

Decade 2010
Started parallelization
Done parallelization


In [8]:
new_index_df.to_parquet(path=f'{save_path}/{args.dec}.parq', engine='fastparquet',compression='snappy',row_group_offsets=50_000_000)


In [7]:
print(f'{new_index_df_2.shape[0]/old_shape}')


NameError: name 'new_index_df_2' is not defined


new_index_df_2=new_index_df.groupby(['lemma_pos','comp_class','is_comp','comp_ner_sent'])['count'].progress_apply(sum).to_frame().reset_index()


In [10]:
new_index_df.head(1_000_000).groupby(['lemma_pos','comp_class','is_comp','comp_ner_sent'],observed=True)['count'].progress_apply(sum).to_frame().reset_index()


  0%|          | 0/999742 [00:00<?, ?it/s]

,lemma_pos,comp_class,is_comp,comp_ner_sent,count
0,'_AUX m_AUX acquaint_VERB with_ADP ._PUNCT,0,False,,1
1,"'_AUX m_AUX afeard_ADJ ._PUNCT ""_PUNCT",0,False,,1
2,"'_AUX m_AUX agree_VERB ._PUNCT ""_PUNCT",0,False,,1
3,'_AUX m_AUX always_ADV fight_VERB ._PUNCT,0,False,,2
4,'_AUX m_AUX badly_ADV wound_VERB ._PUNCT,0,False,,1
...,...,...,...,...,...
999737,word_VERB his_PRON tremble_VERB troop_NOUN bes...,0,False,,9
999738,word_VERB his_PRON way_NOUN ._PUNCT eos_X,0,False,,2
999739,"word_VERB his_PRON will_AUX ,_PUNCT from_ADP",0,False,,4
999740,"word_VERB his_PRON woe_NOUN betray_VERB ,_PUNCT",0,False,,1


In [11]:
new_index_df.head(1_000_000).groupby(['lemma_pos','comp_class','is_comp','comp_ner_sent'],observed=True)['count'].sum().to_frame().reset_index()


,lemma_pos,comp_class,is_comp,comp_ner_sent,count
0,'_AUX m_AUX acquaint_VERB with_ADP ._PUNCT,0,False,,1
1,"'_AUX m_AUX afeard_ADJ ._PUNCT ""_PUNCT",0,False,,1
2,"'_AUX m_AUX agree_VERB ._PUNCT ""_PUNCT",0,False,,1
3,'_AUX m_AUX always_ADV fight_VERB ._PUNCT,0,False,,2
4,'_AUX m_AUX badly_ADV wound_VERB ._PUNCT,0,False,,1
...,...,...,...,...,...
999737,word_VERB his_PRON tremble_VERB troop_NOUN bes...,0,False,,9
999738,word_VERB his_PRON way_NOUN ._PUNCT eos_X,0,False,,2
999739,"word_VERB his_PRON will_AUX ,_PUNCT from_ADP",0,False,,4
999740,"word_VERB his_PRON woe_NOUN betray_VERB ,_PUNCT",0,False,,1
